<a href="https://colab.research.google.com/github/Pratigya-Dhakal/MajorProject-FoodSnap/blob/pratigya/foodsnapmodel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import os
import cv2
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras.utils import to_categorical



In [2]:
#Mount Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# Step 1: Data Preparation

# Read nutrition dataset
nutrition_data = pd.read_csv('/content/drive/MyDrive/foodsnap/nutrition_values.csv')

# Preprocess nutrition dataset as needed



In [4]:
# Define paths to Food-101 dataset
food101_dir = '/content/drive/MyDrive/foodsnap/food-101/images'
food_labels = os.listdir(food101_dir)

# Initialize lists to store image paths and corresponding labels
image_paths = []
labels = []

In [5]:
# Iterate through each food item folder
for label in food_labels:
    food_item_folder = os.path.join(food101_dir, label)
    # Collect image paths from each food item folder
    image_files = os.listdir(food_item_folder)
    food_image_paths = [os.path.join(food_item_folder, image_file) for image_file in image_files]

    # Append image paths and corresponding labels
    image_paths.extend(food_image_paths)
    labels.extend([label] * len(food_image_paths))

In [6]:

# Convert labels to numerical values
label_mapping = {label: index for index, label in enumerate(set(labels))}
numeric_labels = [label_mapping[label] for label in labels]
num_classes = len(set(labels))

In [7]:
# Split the data into training and testing sets
train_image_paths, test_image_paths, train_labels, test_labels = train_test_split(image_paths, numeric_labels, test_size=0.2, random_state=42)



In [8]:
# Step 2: Model Training

# Preprocess the food images
def preprocess_image(image_path):
    img = load_img(image_path, target_size=(224, 224))
    img = img_to_array(img) / 255.0
    return img

In [9]:
# Load pre-trained VGG16 model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the weights of the pre-trained layers
for layer in base_model.layers:
    layer.trainable = False

In [10]:
# Create the model architecture
model = Sequential()
model.add(base_model)
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))


In [11]:
# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [12]:
# Convert labels to one-hot encoded vectors
train_labels_encoded = to_categorical(train_labels, num_classes=num_classes)
test_labels_encoded = to_categorical(test_labels, num_classes=num_classes)

In [13]:
# Train the model
epochs = 3
batch_size = 32

for epoch in range(epochs):
    for batch_start in range(0, len(train_image_paths), batch_size):
        batch_end = batch_start + batch_size
        batch_image_paths = train_image_paths[batch_start:batch_end]
        batch_images = np.array([preprocess_image(image_path) for image_path in batch_image_paths])
        batch_labels = train_labels_encoded[batch_start:batch_end]

        model.train_on_batch(batch_images, batch_labels)

In [14]:
# Evaluate the model
test_images = np.array([preprocess_image(image_path) for image_path in test_image_paths])
accuracy = model.evaluate(test_images, test_labels_encoded)[1]

20/20 [==============================] - 348s 17s/step - loss: 0.5758 - accuracy: 0.7645


In [15]:
# Step 3: Inference and Nutrition Value Retrieval

def predict_food_category(image_path):
    image = preprocess_image(image_path)
    image = np.expand_dims(image, axis=0)
    predicted_probs = model.predict(image)[0]
    predicted_label = np.argmax(predicted_probs)
    return predicted_label




In [16]:

def retrieve_nutrition_values(food_category):
    nutrition_values = nutrition_data[nutrition_data['product_name'] == food_category]
    return nutrition_values

In [17]:
# Example usage
example_image_path = '/content/food.jpg'
predicted_category = predict_food_category(example_image_path)
predicted_food = list(label_mapping.keys())[predicted_category]
nutrition_values = retrieve_nutrition_values(predicted_food)

print('Predicted Food Category:', predicted_food)
print('Nutrition Values:', nutrition_values)

1/1 [==============================] - 1s 693ms/step
Predicted Food Category: pizza
Nutrition Values:    product_name  energy_100g  carbohydrates_100g  sugars_100g  proteins_100g  \
47        pizza    251.12686              26.591        2.262        10.4336   

    fat_100g  fiber_100g  cholesterol_100g  
47   11.2206       1.264           0.02194  


In [18]:
# Example usage
example_image_path = '/content/ham.jpg'
predicted_category = predict_food_category(example_image_path)
predicted_food = list(label_mapping.keys())[predicted_category]
nutrition_values = retrieve_nutrition_values(predicted_food)

print('Predicted Food Category:', predicted_food)
print('Nutrition Values:', nutrition_values)

1/1 [==============================] - 1s 764ms/step
Predicted Food Category: hamburger
Nutrition Values:    product_name  energy_100g  carbohydrates_100g  sugars_100g  proteins_100g  \
33    hamburger      267.441               45.35         4.65            9.3   

    fat_100g  fiber_100g  cholesterol_100g  
33      5.23         2.3               0.0  


In [19]:
# Example usage
example_image_path = '/content/download.jpg'
predicted_category = predict_food_category(example_image_path)
predicted_food = list(label_mapping.keys())[predicted_category]
nutrition_values = retrieve_nutrition_values(predicted_food)

print('Predicted Food Category:', predicted_food)
print('Nutrition Values:', nutrition_values)

1/1 [==============================] - 1s 914ms/step
Predicted Food Category: hamburger
Nutrition Values:    product_name  energy_100g  carbohydrates_100g  sugars_100g  proteins_100g  \
33    hamburger      267.441               45.35         4.65            9.3   

    fat_100g  fiber_100g  cholesterol_100g  
33      5.23         2.3               0.0  


In [20]:
# Example usage
example_image_path = '/content/clinkers-cake-173208-2.jpg'
predicted_category = predict_food_category(example_image_path)
predicted_food = list(label_mapping.keys())[predicted_category]
nutrition_values = retrieve_nutrition_values(predicted_food)

print('Predicted Food Category:', predicted_food)
print('Nutrition Values:', nutrition_values)

1/1 [==============================] - 0s 498ms/step
Predicted Food Category: pizza
Nutrition Values:    product_name  energy_100g  carbohydrates_100g  sugars_100g  proteins_100g  \
47        pizza    251.12686              26.591        2.262        10.4336   

    fat_100g  fiber_100g  cholesterol_100g  
47   11.2206       1.264           0.02194  


In [21]:
# Example usage
example_image_path = '/content/download (1).jpg'
predicted_category = predict_food_category(example_image_path)
predicted_food = list(label_mapping.keys())[predicted_category]
nutrition_values = retrieve_nutrition_values(predicted_food)

print('Predicted Food Category:', predicted_food)
print('Nutrition Values:', nutrition_values)

1/1 [==============================] - 1s 523ms/step
Predicted Food Category: pizza
Nutrition Values:    product_name  energy_100g  carbohydrates_100g  sugars_100g  proteins_100g  \
47        pizza    251.12686              26.591        2.262        10.4336   

    fat_100g  fiber_100g  cholesterol_100g  
47   11.2206       1.264           0.02194  
